# Deep Learning Toolkit for Splunk - Notebook for TensorFlow 2.0
This notebook is based on the **autoencoder** notebook of the standard DSDL containers, and has been modified for educational purposes.
The original code can be found in: [Splunk MLTK Container Docker Github](https://github.com/splunk/splunk-mltk-container-docker)

## Autoencoder Example
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk. As an example we use a custom autoencoder built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries 
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2023-03-21 12:54:42.000038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 12:54:42.112735: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-21 12:54:42.117781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 12:54:42.117795: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.22.1
pandas version: 1.5.1
TensorFlow version: 2.10.0
Keras version: 2.10.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup iris.csv <br>| fit MLTKContainer algo=autoencoder epochs=100 batch_size=4 components=2 petal_length petal_width sepal_length sepal_width into app:iris_autoencoder

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("SC_Autoencoder")
print(df[0:1])
print(df.shape)
print(str(param))

   bytes_received  bytes_sent  packets_received  packets_sent  \
0             170          85                 1             1   

   used_by_malware_yes  used_by_malware_no  
0                    1                   0  
(69510, 6)
{'options': {'params': {'mode': 'stage', 'algo': 'SideChannel_model'}, 'args': ['bytes_received', 'bytes_sent', 'packets_received', 'packets_sent', 'used_by_malware_yes', 'used_by_malware_no'], 'feature_variables': ['bytes_received', 'bytes_sent', 'packets_received', 'packets_sent', 'used_by_malware_yes', 'used_by_malware_no'], 'model_name': 'SC_Autoencoder', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['byt

## Stage 2 - create and initialize a model

In [5]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    components = 4
    components_2 = 2
    activation_fn = 'relu'
    # learning_rate = 0.001
    # epsilon=0.00001 # default 1e-07
    if 'options' in param:
        if 'params' in param['options']:
            if 'components' in param['options']['params']:
                components = int(param['options']['params']['components'])
            if 'activation_func' in param['options']['params']:
                activation_fn = param['options']['params']['activation_func']
            if 'components_2' in param['options']['params']:
                components_2 = int(param['options']['params']['components_2'])
                
    input_shape = int(X.shape[1])
    encoder_layer = keras.layers.Dense(components, input_dim=input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    decoder_layer = keras.layers.Dense(input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    # new layer
    extra_layer = keras.layers.Dense(components_2, activation=activation_fn)
    
    model = keras.Sequential()
    model.add(encoder_layer)
    model.add(extra_layer) # add new layer
    model.add(decoder_layer)
    #opt = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (69510, 6)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 28        
                                                                 
 dense_2 (Dense)             (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 6)                 18        
                                                                 
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
None


2023-03-21 12:54:59.838936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-21 12:54:59.838963: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-21 12:54:59.838982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a95066599963): /proc/driver/nvidia/version does not exist
2023-03-21 12:54:59.839561: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Stage 3 - fit the model

In [7]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    model_epochs = 20
    model_batch_size = 32
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=X, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = X)
    return returns

In [8]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/20
2173/2173 - 2s - loss: 21052524.0000 - accuracy: 0.4984 - 2s/epoch - 888us/step
Epoch 2/20
2173/2173 - 2s - loss: 1061591.6250 - accuracy: 0.7079 - 2s/epoch - 699us/step
Epoch 3/20
2173/2173 - 2s - loss: 765186.6875 - accuracy: 0.7080 - 2s/epoch - 704us/step
Epoch 4/20
2173/2173 - 2s - loss: 764367.5000 - accuracy: 0.7443 - 2s/epoch - 857us/step
Epoch 5/20
2173/2173 - 2s - loss: 764579.0000 - accuracy: 0.7845 - 2s/epoch - 813us/step
Epoch 6/20
2173/2173 - 2s - loss: 763586.6250 - accuracy: 0.8067 - 2s/epoch - 784us/step
Epoch 7/20
2173/2173 - 2s - loss: 765639.6875 - accuracy: 0.8116 - 2s/epoch - 845us/step
Epoch 8/20
2173/2173 - 2s - loss: 770080.6875 - accuracy: 0.8142 - 2s/epoch - 883us/step
Epoch 9/20
2173/2173 - 2s - loss: 761182.3125 - accuracy: 0.8177 - 2s/epoch - 867us/step
Epoch 10/20
2173/2173 - 2s - loss: 761846.0000 - accuracy: 0.8191 - 2s/epoch - 818us/step
Epoch 11/20
2173/2173 - 2s - loss: 761949.0000 - accuracy: 0.8220 - 2s/epoch - 728us/step
Epoch 12/20
2173

## Stage 4 - apply the model

In [9]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    reconstruction = model.predict(x = X)
#     intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.layers[0].output)
#     hidden = intermediate_layer_model.predict(x = X)
#     y_hat = pd.concat([pd.DataFrame(reconstruction).add_prefix("reconstruction_"), pd.DataFrame(hidden).add_prefix("hidden_")], axis=1)

    reconstruction_error = tf.keras.losses.mean_squared_error(X, reconstruction).numpy()
    y_hat = pd.concat([pd.DataFrame(reconstruction_error).add_prefix("reconstructionError_")], axis=1)

    return y_hat

In [10]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

2173/2173 [==============================] - 2s 735us/step
       reconstructionError_0
0                 241.595108
1                 159.410263
2                 169.295898
3              441326.625000
4              135521.484375
...                      ...
69505           73840.789062
69506           32123.349609
69507           42464.121094
69508           42464.121094
69509           42464.121094

[69510 rows x 1 columns]


## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 6 - load the model

In [12]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 7 - provide a summary of the model

In [13]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

In [14]:
summary(model)

{'version': {'tensorflow': '2.10.0', 'keras': '2.10.0'},
 'summary': 'Model: "sequential"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n dense (Dense)               (None, 4)                 28        \n                                                                 \n dense_2 (Dense)             (None, 2)                 10        \n                                                                 \n dense_1 (Dense)             (None, 6)                 18        \n                                                                 \n=================================================================\nTotal params: 56\nTrainable params: 56\nNon-trainable params: 0\n_________________________________________________________________\n'}

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code